<a href="https://colab.research.google.com/github/Amikuto/DAaML/blob/master/06_CNN_embeddings_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [349]:
import pandas as pd
import numpy as np
import nltk
import torch
import torch.nn as nn

# 6. Классификация текстов при помощи сверточных сетей

__Автор__: Никита Владимирович Блохин (NVBlokhin@fa.ru)

Финансовый университет, 2020 г. 

## 1. Представление и предобработка текстовых данных в виде последовательностей

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [350]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [351]:
first_sen = nltk.sent_tokenize(text)[0].replace(".", "").lower()
first_sen

'select your preferences and run the install command'

In [352]:
words = nltk.word_tokenize(first_sen)
words

['select', 'your', 'preferences', 'and', 'run', 'the', 'install', 'command']

In [353]:
words_dict = {k: v for v, k in enumerate(words)}
words_dict

{'select': 0,
 'your': 1,
 'preferences': 2,
 'and': 3,
 'run': 4,
 'the': 5,
 'install': 6,
 'command': 7}

In [354]:
[words_dict[i] for i in words]

[0, 1, 2, 3, 4, 5, 6, 7]

1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [355]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [356]:
all_words = nltk.word_tokenize(text.lower().replace(".", ""))
all_words_dict = {k: v for v, k in enumerate(all_words)}
all_words_dict

{'select': 0,
 'your': 1,
 'preferences': 2,
 'and': 14,
 'run': 4,
 'the': 10,
 'install': 6,
 'command': 7,
 'stable': 8,
 'represents': 9,
 'most': 11,
 'currently': 12,
 'tested': 13,
 'supported': 15,
 'version': 16,
 'of': 17,
 'pytorch': 18,
 'note': 19,
 'that': 20,
 'libtorch': 21,
 'is': 22,
 'only': 23,
 'available': 24,
 'for': 25,
 'c++': 26}

In [357]:
first_sen = nltk.sent_tokenize(text)[0].replace(".", "").lower()
first_sen_words = nltk.word_tokenize(first_sen)
first_sen_words

['select', 'your', 'preferences', 'and', 'run', 'the', 'install', 'command']

In [358]:
tensor = torch.zeros(len(first_sen_words), len(all_words_dict))

In [359]:
for i, word in enumerate(first_sen_words):
  tensor[i][all_words_dict[word]] = 1
  print(all_words_dict[word])

0
1
2
14
4
10
6
7


In [360]:
tensor

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.]])

In [361]:
tensor.shape

torch.Size([8, 25])

1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [362]:
from collections import Counter

vocab = Counter(all_words_dict)
vocab = sorted(vocab, key=vocab.get, reverse=True)
vocab_size = len(vocab)
vocab_size

25

In [363]:
word2idx = {word: ind for ind, word in enumerate(vocab)}
word2idx

{'c++': 0,
 'for': 1,
 'available': 2,
 'only': 3,
 'is': 4,
 'libtorch': 5,
 'that': 6,
 'note': 7,
 'pytorch': 8,
 'of': 9,
 'version': 10,
 'supported': 11,
 'and': 12,
 'tested': 13,
 'currently': 14,
 'most': 15,
 'the': 16,
 'represents': 17,
 'stable': 18,
 'command': 19,
 'install': 20,
 'run': 21,
 'preferences': 22,
 'your': 23,
 'select': 24}

In [364]:
encoded_sentences = [word2idx[word] for word in first_sen_words]
encoded_sentences

[24, 23, 22, 12, 21, 16, 20, 19]

In [365]:
emb_dim = 25
emb_layer = nn.Embedding(vocab_size, emb_dim)
word_vectors = emb_layer(torch.LongTensor(encoded_sentences))
word_vectors

tensor([[-9.8497e-01, -8.3152e-01, -1.5091e+00, -1.1183e+00,  5.6081e-02,
         -7.6570e-01,  1.0942e+00,  6.2499e-01,  7.0286e-01, -2.7504e-01,
          9.7120e-01, -5.0217e-01, -2.4522e-01,  1.0535e+00,  2.0797e+00,
          4.4264e-01, -1.2697e+00,  7.3438e-01, -7.2332e-01,  1.0820e+00,
         -2.4006e-01, -1.8046e+00,  9.3524e-01,  8.2588e-01, -5.5058e-01],
        [-1.2424e+00, -1.1253e+00, -6.5582e-02,  2.2343e-01,  1.3020e+00,
         -1.2041e+00,  2.4841e-01,  1.2204e-01,  1.0141e+00, -2.8781e-01,
          1.5336e-02,  4.4684e-01,  3.6950e-01, -1.2953e+00, -5.6597e-02,
          2.7682e-01, -4.4070e-01, -8.0875e-01, -9.1628e-01, -1.7093e+00,
         -1.4255e-01,  1.0989e+00, -1.0450e+00,  1.0068e+00,  4.8541e-01],
        [-4.4633e-02, -8.7612e-01, -2.3910e+00, -2.0590e-01,  7.9070e-01,
         -6.6066e-01,  9.0151e-01,  3.5316e-01, -6.2291e-01,  7.4781e-01,
          6.2062e-01,  2.3577e+00, -1.4266e-01,  5.8851e-02,  8.0076e-01,
         -2.0228e+00,  6.5720e-01,  

In [366]:
word_vectors.shape

torch.Size([8, 25])

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса> 
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта: 
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [367]:
surname_dataset = pd.read_csv("./surnames/surnames.csv")
surname_dataset.head()

,surname,nationality
0,Woodford,English
1,Coté,French
2,Kore,English
3,Koury,Arabic
4,Lebzak,Russian


In [368]:
surname_dict = pd.Series(surname_dataset.nationality.unique()).to_dict()
surname_dict = dict(map(reversed, surname_dict.items()))
surname_dict

{'English': 0,
 'French': 1,
 'Arabic': 2,
 'Russian': 3,
 'Japanese': 4,
 'Chinese': 5,
 'Italian': 6,
 'Czech': 7,
 'Irish': 8,
 'German': 9,
 'Greek': 10,
 'Spanish': 11,
 'Polish': 12,
 'Dutch': 13,
 'Vietnamese': 14,
 'Korean': 15,
 'Portuguese': 16,
 'Scottish': 17}

In [369]:
dataset_nation_as_index = surname_dataset.copy()
dataset_nation_as_index.nationality = surname_dataset.nationality.map(lambda x: surname_dict[x])
dataset_nation_as_index

,surname,nationality
0,Woodford,0
1,Coté,1
2,Kore,0
3,Koury,2
4,Lebzak,3
...,...,...
10975,Quraishi,2
10976,Innalls,0
10977,Król,12
10978,Purvis,0


In [370]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset_nation_as_index["surname"], dataset_nation_as_index["nationality"], test_size=0.2)

In [371]:
X_train

3163           Toma
2232      Mokrinsky
1150         Ohishi
7923        Kumiega
10284        Durnev
            ...    
8497        Krawiec
453      Jachmenkov
6166       Gasyukov
5181      Ratcliffe
7635       Reinders
Name: surname, Length: 8784, dtype: object

In [372]:
max(X_train.map(len))

17

In [373]:
y_train

3163      2
2232      3
1150      4
7923     12
10284     3
         ..
8497      7
453       3
6166      3
5181      0
7635     13
Name: nationality, Length: 8784, dtype: int64

In [374]:
class Vocab:
  def __init__(self, column: pd.DataFrame | pd.Series):
    all_chars = pd.Series(column.values).map(lambda x: list(x.lower())).explode().unique()
    all_chars = np.insert(all_chars, 0, "<PAD>")
    self.idx_to_token = {index: token for index, token in enumerate(all_chars)}
    self.token_to_idx = {token: index for index, token in enumerate(all_chars)}
    self.max_seq_len = max(column.map(len))

In [375]:
vocab = Vocab(dataset_nation_as_index["surname"])
vocab.max_seq_len

17

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса>
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

In [382]:
from torch.utils.data import Dataset


class SurnamesDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab
    self.max_X = 17
    self.max_y = 10

  def vectorize(self, surename):
    tensor = torch.zeros(self.vocab.max_seq_len, dtype=torch.long)
    for i, val in enumerate(surename):
      tensor[i] = self.vocab.token_to_idx[val.lower()]
    tensor[tensor==0] = self.vocab.token_to_idx["<PAD>"]
    return tensor

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    surname = self.X.iloc[idx]

    nation = self.y.iloc[idx]

    return self.vectorize(surname).tolist(), nation

In [383]:
dataset = SurnamesDataset(X=X_train, y=y_train, vocab=vocab)

In [384]:
X_train.iloc[0], y_train.iloc[0]

('Toma', 2)

In [385]:
dataset[0][0]

[7, 2, 20, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]



2.6. Обучить классификатор.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта:
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

In [406]:
l = nn.Embedding(21, 17)
w = l(torch.LongTensor(dataset[0][0]))
w

tensor([[ 0.9094,  1.3261,  1.2423, -0.0236, -0.7022,  0.6126, -0.6979,  0.4125,
         -0.5350, -0.0808, -0.3761, -1.1356,  0.7009, -0.6870,  0.6529,  0.4226,
         -0.6821],
        [-0.6040, -0.3058, -1.0245,  2.1413, -0.0207, -0.1976,  0.7568, -0.0139,
         -0.0611, -1.3718,  2.1177,  1.4276,  1.3494,  0.4159,  1.5932, -0.2202,
         -1.4642],
        [ 0.4182,  0.5255,  0.3523,  1.2183,  0.9574, -1.4453,  0.0061,  1.6689,
         -0.3007,  0.5705,  0.0977, -0.0649, -0.0423,  0.1966,  0.7472, -1.4083,
         -0.4024],
        [-0.2523, -0.0890,  0.9858,  1.1221,  0.3944,  1.0503, -1.5702, -1.6974,
          1.6146, -2.1063, -1.7663, -0.6850,  0.3767,  0.5437, -0.5237,  1.1585,
         -1.2444],
        [-0.6220, -1.2674,  1.1593,  0.0367,  0.4477,  0.2179, -1.3347, -0.0228,
         -0.5904, -0.9591, -1.1340,  1.9351, -0.0723, -0.6129,  0.1259, -0.2515,
          0.7828],
        [-0.6220, -1.2674,  1.1593,  0.0367,  0.4477,  0.2179, -1.3347, -0.0228,
         -0.59

In [405]:
len(dataset[0][0])

17

## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding` 
    - подберите адекватную размерность вектора эмбеддинга: 
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%

In [ ]:
positive_raw = pd.read_csv("./polarity/positive_reviews.csv", header=None)
negative_raw = pd.read_csv("./polarity/negative_reviews.csv", header=None)

In [ ]:
positive_raw.head()

In [ ]:
negative_raw.head()

In [ ]:
positive_raw["state"] = 1
negative_raw["state"] = 0

In [ ]:
negative_raw

In [ ]:
positive_raw

In [ ]:
all_reviews = pd.concat([negative_raw, positive_raw], axis=0)
all_reviews.columns = ["review", "rating"]
all_reviews

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(all_reviews["review"], all_reviews["rating"], test_size=0.2)

In [ ]:
nltk.word_tokenize(all_reviews.review)

In [ ]:
all_reviews["review"].flatten()